<a href="https://colab.research.google.com/github/Chandru-018/Chandrasekhar_INFO5731_FALL2024/blob/main/Karumanchi_Chandrasekhar_Exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 4**

**This exercise will provide a valuable learning experience in working with text data and extracting features using various topic modeling algorithms. Key concepts such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA), lda2vec, and BERTopic.**

***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)

**Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
import re
from gensim.utils import simple_preprocess
import spacy

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
# Import Dataset
# Fetch the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# Extract the content
data = newsgroups.data

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Function to compute coherence score
def compute_coherence_score(corpus, texts, dictionary, k):
    lda_model = gensim.models.LdaModel(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=k,
                                       random_state=42,
                                       passes=10,
                                       chunksize=100,  # Adjust chunksize
                                       per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

## Question 2 (10 Points)

**Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [15]:
pip install gensim scikit-learn nltk

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample data
documents = ["Your text data goes here."]  # Replace with your data

# Create the Document-Term Matrix
vectorizer = TfidfVectorizer(stop_words='english')
dtm = vectorizer.fit_transform(documents)


In [17]:
from sklearn.decomposition import TruncatedSVD

# Choose the number of components/topics
# num_topics should be less than or equal to the number of features in the DTM
num_topics = min(3, dtm.shape[1]) # set to 3 or less based on the number of features

lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
lsa_topic_matrix = lsa_model.fit_transform(dtm)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_truncated_svd.py:275: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


In [18]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
import gensim

def compute_coherence_score(lda_model, texts, dictionary):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

# Create a dictionary and corpus for Gensim
dictionary = Dictionary([doc.split() for doc in documents])
corpus = [dictionary.doc2bow(doc.split()) for doc in documents]

# Find the best number of topics by coherence score
coherence_scores = []
# Change here: Ensure that the maximum number of topics (k) is less than or equal to the number of features in the DTM.
# You can access this as dtm.shape[1] or len(vectorizer.get_feature_names_out())
max_topics = min(20, dtm.shape[1])  # Set

In [19]:
import numpy as np

def print_topics(lsa_model, vectorizer, n_words=5):
    terms = vectorizer.get_feature_names_out()
    for i, topic in enumerate(lsa_model.components_):
        top_indices = topic.argsort()[-n_words:][::-1]
        print(f"Topic {i + 1}: " + ", ".join(terms[top_indices]))

print_topics(lsa_model, vectorizer)


Topic 1: text, goes, data


In [26]:
pip install lda2vec numpy pandas nltk


  Preparing metadata (setup.py) ... done
  Created wheel for lda2vec: filename=lda2vec-0.16.10-py3-none-any.whl size=14406 sha256=b954f887032284b529a65d29058d127c3500ca0ca3c8767a65be1fd51e8077ff
  Stored in directory: /root/.cache/pip/wheels/1e/90/24/a97126c0fe8b479ba3bb79d3b18ebaab571a18d90bb2967ab6
Successfully built lda2vec


In [30]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# Load the dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

# Preprocess the data
stop_words = set(stopwords.words('english'))
preprocessed_docs = []

for doc in documents:
    tokens = word_tokenize(doc.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    preprocessed_docs.append(filtered_tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.4 MB/s eta 0:00:00


In [35]:
!pip install keras-preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [37]:
!pip install tensorflow

## Question 4 (10 points):
**Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [39]:
pip install bertopic transformers nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.2 MB/s eta 0:00:00


In [40]:
import pandas as pd
import nltk
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

# Preprocess the data (optional: can be refined)
preprocessed_docs = []
for doc in documents:
    tokens = doc.lower().split()
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    preprocessed_docs.append(' '.join(filtered_tokens))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
from bertopic import BERTopic

# Initialize BERTopic
topic_model = BERTopic()

# Fit the model
topics, probabilities = topic_model.fit_transform(preprocessed_docs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from bertopic import BERTopic
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

# Initialize BERTopic
topic_model = BERTopic()

# Fit the model # Make sure this line is executed before calling `get_topic_info()`
topics, probabilities = topic_model.fit_transform(preprocessed_docs)

# Get topic words
topic_words = topic_model.get_topic_info()

# Prepare data for coherence score calculation
texts = [doc.split() for doc in preprocessed_docs]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Function to compute coherence score
def compute_coherence_score(model, texts, dictionary, k):
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

# Assess coherence score for each topic and find the optimal K
best_k = None
best_coherence = -1

for k in range(2, 21):  # Adjust the range as needed
    # Create a new topic model for each k
    temp_topic_model = BERTopic(nr_topics=k)
    topics, _ = temp_topic_model.fit_transform(preprocessed_docs) #Fit the temporary model
    coherence_score = compute_coherence_score(temp_topic_model, texts, dictionary, k) # Pass the temporary model here

    if coherence_score > best_coherence:
        best_coherence = coherence_score
        best_k = k

print(f"Best number of topics: {best_k} with coherence score: {best_coherence}")

## **Question 3 (Alternative) - (10 points)**

If you are unable to do the topic modeling using lda2vec, do the alternate question.

Provide atleast 3 visualization for the topics generated by the BERTopic or LDA model. Explain each of the visualization in detail.

In [ ]:
import pandas as pd
import nltk
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from bertopic import BERTopic
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


# Download stopwords if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

# Preprocess the data (optional: can be refined)
preprocessed_docs = []
for doc in documents:
    tokens = doc.lower().split()
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    preprocessed_docs.append(' '.join(filtered_tokens))

# Prepare data for coherence score calculation
texts = [doc.split() for doc in preprocessed_docs]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Function to compute coherence score
def compute_coherence_score(model, texts, dictionary, k):
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

# Assess coherence score for each topic and find the optimal K
best_k = None
best_coherence = -1
topic_model = None  # Initialize topic_model to None

for k in range(2, 21):  # Adjust the range as needed
    # Create a new topic model for each k
    temp_topic_model = BERTopic(nr_topics=k)
    topics, _ = temp_topic_model.fit_transform(preprocessed_docs) #Fit the temporary model
    coherence_score = compute_coherence_score(temp_topic_model, texts, dictionary, k) # Pass the temporary model here

    if coherence_score > best_coherence:
        best_coherence = coherence_score
        best_k = k
        topic_model = temp_topic_model

print(f"Best number of topics: {best_k} with coherence score: {best_coherence}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00


## Extra Question (5 Points)

**Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.**

**This question will compensate for any points deducted in this exercise. Maximum marks for the exercise is 40 points.**

In [ ]:
# Write your code here


# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.

Consider the following points in your response:

**Learning Experience:** Describe your overall learning experience in working with text data and extracting features using various topic modeling algorithms. Did you understand these algorithms and did the implementations helped in grasping the nuances of feature extraction from text data.

**Challenges Encountered:** Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
This assignment helped me in understanding the Bart, Bert and LDA. Moreover it is helpful for my project aswell.





'''